<a href="https://colab.research.google.com/github/arnabbiswas66/multimodal-fake-news-classifier/blob/main/Twitter_multimodal_classifier_Scaled_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#!pip install -q -U "tensorflow-text==2.11.*"
!pip install -q tensorflow_text
!pip install -q talos

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 82.5 MB/s eta 0:00:00


In [3]:
from tensorflow.python.platform.tf_logging import warn
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import os
import re
from os import listdir
import shutil
import glob
import random
import matplotlib.pyplot as plt
import talos as ta
import tensorflow_hub as hub
import tensorflow_text as text
tf.get_logger().setLevel('ERROR')
import warnings

warnings.filterwarnings('ignore')

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
BASE_SAVE_LOCATION = "/content/drive/MyDrive/multimodal-news"
CHECKPOINT_FILEPATH = '/content/drive/MyDrive/multimodal-news/twitter-models/'

## Load the dataframes

In [6]:
#Load training df
train_df_model = pd.read_pickle(BASE_SAVE_LOCATION+"/twitter-train.pickle")
test_df_model = pd.read_pickle(BASE_SAVE_LOCATION+"/twitter-test.pickle")
print(train_df_model.shape, test_df_model.shape)

(14258, 3) (1923, 3)


In [7]:
# This is for Talos as it doesn't take TF DataSet as input
X_train = train_df_model.copy()
y_train = X_train.pop('label')

X_test = test_df_model.copy()
y_test = X_test.pop('label')

# Data input pipeline

In [8]:
# Define TF Hub paths to the BERT encoder and its preprocessor
bert_model_path = (
    "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1"
)
bert_preprocess_path = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"

In [9]:
def make_bert_preprocessing_model(sentence_features, seq_length=128):
    """Returns Model mapping string features to BERT inputs.

  Args:
    sentence_features: A list with the names of string-valued features.
    seq_length: An integer that defines the sequence length of BERT inputs.

  Returns:
    A Keras Model that can be called on a list or dict of string Tensors
    (with the order or names, resp., given by sentence_features) and
    returns a dict of tensors for input to BERT.
  """

    input_segments = [
        tf.keras.layers.Input(shape=(), dtype=tf.string, name=ft)
        for ft in sentence_features
    ]

    # Tokenize the text to word pieces.
    bert_preprocess = hub.load(bert_preprocess_path)
    tokenizer = hub.KerasLayer(bert_preprocess.tokenize, name="tokenizer")
    segments = [tokenizer(s) for s in input_segments]

    # Optional: Trim segments in a smart way to fit seq_length.
    # Simple cases (like this example) can skip this step and let
    # the next step apply a default truncation to approximately equal lengths.
    truncated_segments = segments

    # Pack inputs. The details (start/end token ids, dict of output tensors)
    # are model-dependent, so this gets loaded from the SavedModel.
    packer = hub.KerasLayer(
        bert_preprocess.bert_pack_inputs,
        arguments=dict(seq_length=seq_length),
        name="packer",
    )
    model_inputs = packer(truncated_segments)
    return keras.Model(input_segments, model_inputs)


In [10]:
bert_preprocess_model = make_bert_preprocessing_model(["text_1"])
bert_preprocess_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_1 (InputLayer)         [(None,)]                 0         
                                                                 
 tokenizer (KerasLayer)      (None, None, None)        0         
                                                                 
 packer (KerasLayer)         {'input_word_ids': (None  0         
                             , 128),                             
                              'input_mask': (None, 12            
                             8),                                 
                              'input_type_ids': (None            
                             , 128)}                             
                                                                 
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [11]:
idx = np.random.choice(len(train_df_model))
row = train_df_model.iloc[idx]
sample_text_1, sample_text_2 = row["tweetText"], row["tweetText"]
print(f"Text 1: {sample_text_1}")
print(f"Text 2: {sample_text_2}")

test_text = [np.array([sample_text_1])]
text_preprocessed = bert_preprocess_model(test_text)

print("Keys           : ", list(text_preprocessed.keys()))
print("Shape Word Ids : ", text_preprocessed["input_word_ids"].shape)
print("Word Ids       : ", text_preprocessed["input_word_ids"][0, :16])
print("Shape Mask     : ", text_preprocessed["input_mask"].shape)
print("Input Mask     : ", text_preprocessed["input_mask"][0, :16])
print("Shape Type Ids : ", text_preprocessed["input_type_ids"].shape)
print("Type Ids       : ", text_preprocessed["input_type_ids"][0, :16])


Text 1: This is Breezy Point, Queens as seen by @TomKaminskiWCBS in Chopper 880 this morning. http://t.co/kt8o8NwS #Sandy http://t.co/NRnW8O0s
Text 2: This is Breezy Point, Queens as seen by @TomKaminskiWCBS in Chopper 880 this morning. http://t.co/kt8o8NwS #Sandy http://t.co/NRnW8O0s
Keys           :  ['input_word_ids', 'input_mask', 'input_type_ids']
Shape Word Ids :  (1, 128)
Word Ids       :  tf.Tensor(
[  101  2023  2003 21986  9096  2391  1010  8603  2004  2464  2011  1030
  3419 27052 19880 16526], shape=(16,), dtype=int32)
Shape Mask     :  (1, 128)
Input Mask     :  tf.Tensor([1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1], shape=(16,), dtype=int32)
Shape Type Ids :  (1, 128)
Type Ids       :  tf.Tensor([0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0], shape=(16,), dtype=int32)


In [12]:
def dataframe_to_dataset(dataframe):
    columns = ['tweetText', 'image_1', 'label']
    dataframe = dataframe[columns].copy()
    labels = dataframe.pop("label")
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    ds = ds.shuffle(buffer_size=len(dataframe))
    return ds

## Preprocessing utilities

In [13]:
resize = (224, 224)
bert_input_features = ["input_word_ids", "input_type_ids", "input_mask"]

In [14]:
def preprocess_image(image_path):
  extension = tf.strings.split(image_path,'.')[-1]
  image = tf.io.read_file(image_path)
  if extension == b"gif":
    image = tf.io.decode_image(image, 3, expand_animations=False)
  elif extension == b"png":
    image = tf.image.decode_png(image, 3)
  else:
    image = tf.image.decode_jpeg(image, 3)
  image = tf.image.resize(image, resize)
  return image

In [15]:
def preprocess_text(text_1):
  text_1 = tf.convert_to_tensor([text_1])
  output = bert_preprocess_model([text_1])
  output = {feature: tf.squeeze(output[feature]) for feature in bert_input_features}
  return output

In [16]:
def preprocess_text_and_image(sample):
  image_1 = preprocess_image(sample["image_1"])
  text = preprocess_text(sample["tweetText"])
  return {"image_1": image_1, "text": text}

In [17]:
batch_size = 32
auto = tf.data.AUTOTUNE

In [18]:
def prepare_dataset(dataframe, training=True):
  ds = dataframe_to_dataset(dataframe)
  if training:
      ds = ds.shuffle(len(train_df_model))
  ds = ds.map(lambda x, y: (preprocess_text_and_image(x), y)).cache()
  ds = ds.batch(batch_size).prefetch(auto)
  return ds

In [19]:
train_ds = prepare_dataset(train_df_model)
test_ds = prepare_dataset(test_df_model, False)

## Model Building

### Projection

In [20]:
def project_embeddings(
    embeddings, num_projection_layers, projection_dims, dropout_rate
):
    projected_embeddings = keras.layers.Dense(units=projection_dims)(embeddings)
    for _ in range(num_projection_layers):
        x = tf.nn.gelu(projected_embeddings)
        x = keras.layers.Dense(projection_dims)(x)
        x = keras.layers.Dropout(dropout_rate)(x)
        x = keras.layers.Add()([projected_embeddings, x])
        projected_embeddings = keras.layers.LayerNormalization()(x)
    return projected_embeddings


### Vision encoder

In [21]:
def create_vision_encoder(
    num_projection_layers, projection_dims, dropout_rate, trainable=False
):
    # Load the pre-trained ResNet50V2 model to be used as the base encoder.
    resnet_v2 = keras.applications.EfficientNetV2B3(
        include_top=False, weights="imagenet", pooling="avg"
    )
    # Set the trainability of the base encoder.
    for layer in resnet_v2.layers:
        layer.trainable = trainable

    # Receive the images as inputs.
    image_1 = keras.Input(shape=(224, 224, 3), name="image_1")
    
    # Preprocess the input image.
    preprocessed_1 = keras.applications.resnet_v2.preprocess_input(image_1)
    
    # Generate the embeddings for the images using the resnet_v2 model
    # concatenate them.
    embeddings = resnet_v2(preprocessed_1)
    #embeddings = keras.layers.Concatenate()([embeddings_1, embeddings_2])

    # Project the embeddings produced by the model.
    outputs = project_embeddings(
        embeddings, num_projection_layers, projection_dims, dropout_rate
    )
    # Create the vision encoder model.
    return keras.Model([image_1], outputs, name="vision_encoder")

### Text Encoder

In [22]:
def create_text_encoder(
    num_projection_layers, projection_dims, dropout_rate, trainable=False
):
    # Load the pre-trained BERT model to be used as the base encoder.
    bert = hub.KerasLayer(bert_model_path, name="bert",)
    # Set the trainability of the base encoder.
    bert.trainable = trainable

    # Receive the text as inputs.
    bert_input_features = ["input_type_ids", "input_mask", "input_word_ids"]
    inputs = {
        feature: keras.Input(shape=(128,), dtype=tf.int32, name=feature)
        for feature in bert_input_features
    }

    # Generate embeddings for the preprocessed text using the BERT model.
    embeddings = bert(inputs)["pooled_output"]

    # Project the embeddings produced by the model.
    outputs = project_embeddings(
        embeddings, num_projection_layers, projection_dims, dropout_rate
    )
    # Create the text encoder model.
    return keras.Model(inputs, outputs, name="text_encoder")

### Multi Head Attention Layer

In [23]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, x, y, training):
        attn_output = self.att(x, y)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

## MultiModal model

In [24]:
def create_multimodal_model(
    num_projection_layers=0,
    projection_dims=224,
    dropout_rate=0.1,
    vision_trainable=False,
    text_trainable=False,
    attention=False
):
    # Receive the images as inputs.
    image_1 = keras.Input(shape=(224, 224, 3), name="image_1")
    
    # Receive the text as inputs.
    bert_input_features = ["input_type_ids", "input_mask", "input_word_ids"]
    text_inputs = {
        feature: keras.Input(shape=(128,), dtype=tf.int32, name=feature)
        for feature in bert_input_features
    }

    # Create the encoders.
    vision_encoder = create_vision_encoder(
        num_projection_layers, projection_dims, dropout_rate, vision_trainable
    )
    text_encoder = create_text_encoder(
        num_projection_layers, projection_dims, dropout_rate, text_trainable
    )

    # Fetch the embedding projections.
    vision_projections = vision_encoder([image_1])
    vision_projections = keras.layers.Dropout(dropout_rate)(vision_projections)
    text_projections = text_encoder(text_inputs)
    text_projections = keras.layers.Dropout(dropout_rate)(text_projections)
    
    # Cross-attention.
    if attention:
      transformer_block = TransformerBlock(projection_dims, 4, projection_dims)
      x = transformer_block(tf.expand_dims(vision_projections, -1), tf.expand_dims(text_projections, -1))
      x = tf.keras.layers.Flatten()(x)
      
    # Concatenate the projections and pass through the classification layer.
    concatenated = keras.layers.Concatenate()([vision_projections, text_projections])
    if attention:
        concatenated = keras.layers.Concatenate()([concatenated, x])
        #x = tf.keras.layers.Flatten(x)
    outputs = keras.layers.Dense(1, activation="sigmoid")(concatenated)
    return keras.Model([image_1, text_inputs], outputs)

In [25]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics= [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

## Final Multimodal model

In [26]:
def multimodal_model(X_train, y_train, X_test, y_test, params):
  train_ds = prepare_dataset(train_df_model)
  test_ds = prepare_dataset(test_df_model, False)
  model = create_multimodal_model(params['num_projection_layers'],
    params['projection_dims'],
    params['dropout_rate'],
    params['vision_trainable'],
    params['text_trainable'],
    params['attention'])
  model.compile(
    optimizer=tf.keras.optimizers.Adam(params['lr']), loss=loss, metrics=metrics
  )
  history = model.fit(
      train_ds, validation_data=test_ds, 
      epochs=params['epochs'], batch_size=params['batch_size'])
  return history, model
  

In [27]:
params = {
    'num_projection_layers' : [0],
    'projection_dims' : [128, 224],
    'dropout_rate' : [0.1, 0.2],
    'vision_trainable' : [False],
    'text_trainable' : [False],
    'attention' : [True],
    'lr' : [0.001, 0.0005],
    'epochs' : [10],
    'batch_size' : [32, 64]
}

In [ ]:
h = ta.Scan(x = X_train, y= y_train, params = params, model = multimodal_model, x_val = X_test, y_val = y_test, experiment_name = 'multi1', save_weights=False, print_params=True)

  0%|          | 0/16 [00:00<?, ?it/s]

{'num_projection_layers': 0, 'projection_dims': 128, 'dropout_rate': 0.1, 'vision_trainable': False, 'text_trainable': False, 'attention': True, 'lr': 0.001, 'epochs': 10, 'batch_size': 32}
52606240/52606240 [==============================] - 3s 0us/step
Epoch 1/10
446/446 [==============================] - 383s 763ms/step - loss: 0.5123 - accuracy: 0.7680 - precision: 0.7019 - recall: 0.5660 - val_loss: 0.4329 - val_accuracy: 0.8294 - val_precision: 0.8347 - val_recall: 0.9080
Epoch 2/10
446/446 [==============================] - 52s 117ms/step - loss: 0.3573 - accuracy: 0.8514 - precision: 0.8119 - recall: 0.7392 - val_loss: 0.7081 - val_accuracy: 0.5642 - val_precision: 0.9165 - val_recall: 0.3364
Epoch 3/10
446/446 [==============================] - 53s 118ms/step - loss: 0.3124 - accuracy: 0.8681 - precision: 0.8357 - recall: 0.7674 - val_loss: 0.5549 - val_accuracy: 0.7639 - val_precision: 0.9414 - val_recall: 0.6653
Epoch 4/10
446/446 [==============================] - 53s 120ms

  6%|▋         | 1/16 [15:08<3:47:14, 908.97s/it]

{'num_projection_layers': 0, 'projection_dims': 128, 'dropout_rate': 0.1, 'vision_trainable': False, 'text_trainable': False, 'attention': True, 'lr': 0.001, 'epochs': 10, 'batch_size': 64}
Epoch 1/10
446/446 [==============================] - 105s 200ms/step - loss: 0.4749 - accuracy: 0.7901 - precision: 0.7823 - recall: 0.6157 - val_loss: 0.5123 - val_accuracy: 0.8248 - val_precision: 0.9207 - val_recall: 0.7887
Epoch 2/10
446/446 [==============================] - 53s 120ms/step - loss: 0.3414 - accuracy: 0.8578 - precision: 0.8203 - recall: 0.7511 - val_loss: 0.5712 - val_accuracy: 0.7239 - val_precision: 0.9322 - val_recall: 0.6040
Epoch 3/10
446/446 [==============================] - 54s 120ms/step - loss: 0.2996 - accuracy: 0.8792 - precision: 0.8494 - recall: 0.7886 - val_loss: 0.5979 - val_accuracy: 0.7077 - val_precision: 0.9436 - val_recall: 0.5684
Epoch 4/10
446/446 [==============================] - 54s 121ms/step - loss: 0.2750 - accuracy: 0.8870 - precision: 0.8565 - rec

 12%|█▎        | 2/16 [25:08<2:49:36, 726.87s/it]

...

{'num_projection_layers': 0, 'projection_dims': 128, 'dropout_rate': 0.2, 'vision_trainable': False, 'text_trainable': False, 'attention': True, 'lr': 0.001, 'epochs': 10, 'batch_size': 64}
Epoch 1/10
446/446 [==============================] - 108s 205ms/step - loss: 0.4855 - accuracy: 0.7841 - precision: 0.7619 - recall: 0.6230 - val_loss: 0.7174 - val_accuracy: 0.4815 - val_precision: 0.8777 - val_recall: 0.2022
Epoch 2/10
446/446 [==============================] - 53s 120ms/step - loss: 0.3522 - accuracy: 0.8506 - precision: 0.8087 - recall: 0.7411 - val_loss: 0.5419 - val_accuracy: 0.7832 - val_precision: 0.9399 - val_recall: 0.6993
Epoch 3/10
446/446 [==============================] - 54s 121ms/step - loss: 0.3105 - accuracy: 0.8740 - precision: 0.8402 - recall: 0.7825 - val_loss: 0.6314 - val_accuracy: 0.6599 - val_precision: 0.9396 - val_recall: 0.4896
Epoch 4/10
446/446 [==============================] - 54s 121ms/step - loss: 0.2902 - accuracy: 0.8796 - precision: 0.8467 - rec

 38%|███▊      | 6/16 [1:07:07<1:47:19, 643.98s/it]

{'num_projection_layers': 0, 'projection_dims': 128, 'dropout_rate': 0.2, 'vision_trainable': False, 'text_trainable': False, 'attention': True, 'lr': 0.0005, 'epochs': 10, 'batch_size': 32}
Epoch 1/10
446/446 [==============================] - 107s 204ms/step - loss: 0.5021 - accuracy: 0.7674 - precision: 0.7512 - recall: 0.5746 - val_loss: 0.5294 - val_accuracy: 0.8196 - val_precision: 0.9526 - val_recall: 0.7498
Epoch 2/10
446/446 [==============================] - 53s 119ms/step - loss: 0.3658 - accuracy: 0.8468 - precision: 0.8144 - recall: 0.7182 - val_loss: 0.6870 - val_accuracy: 0.5751 - val_precision: 0.9167 - val_recall: 0.3554
Epoch 3/10
446/446 [==============================] - 54s 121ms/step - loss: 0.3340 - accuracy: 0.8585 - precision: 0.8221 - recall: 0.7511 - val_loss: 0.5951 - val_accuracy: 0.6698 - val_precision: 0.9320 - val_recall: 0.5112
Epoch 4/10
446/446 [==============================] - 54s 121ms/step - loss: 0.3087 - accuracy: 0.8701 - precision: 0.8371 - re

 44%|████▍     | 7/16 [1:17:09<1:34:31, 630.12s/it]

{'num_projection_layers': 0, 'projection_dims': 128, 'dropout_rate': 0.2, 'vision_trainable': False, 'text_trainable': False, 'attention': True, 'lr': 0.0005, 'epochs': 10, 'batch_size': 64}
Epoch 1/10
446/446 [==============================] - 111s 215ms/step - loss: 0.5366 - accuracy: 0.7535 - precision: 0.7246 - recall: 0.5605 - val_loss: 0.4396 - val_accuracy: 0.8326 - val_precision: 0.9403 - val_recall: 0.7829
Epoch 2/10
446/446 [==============================] - 54s 120ms/step - loss: 0.3821 - accuracy: 0.8332 - precision: 0.7943 - recall: 0.6952 - val_loss: 0.4335 - val_accuracy: 0.8315 - val_precision: 0.8409 - val_recall: 0.9022
Epoch 3/10
446/446 [==============================] - 54s 121ms/step - loss: 0.3442 - accuracy: 0.8565 - precision: 0.8190 - recall: 0.7482 - val_loss: 0.4249 - val_accuracy: 0.8310 - val_precision: 0.8382 - val_recall: 0.9056
Epoch 4/10
446/446 [==============================] - 55s 122ms/step - loss: 0.3145 - accuracy: 0.8664 - precision: 0.8295 - re

In [24]:
params = {
    'num_projection_layers' : [1],
    'projection_dims' : [128, 224],
    'dropout_rate' : [0.1, 0.2],
    'vision_trainable' : [False],
    'text_trainable' : [False],
    'attention' : [True],
    'lr' : [0.001, 0.0005],
    'epochs' : [10],
    'batch_size' : [32]
}

In [ ]:
h = ta.Scan(x = X_train, y= y_train, params = params, model = multimodal_model, x_val = X_test, y_val = y_test, experiment_name = 'multi1', save_weights=False, print_params=True)

  0%|          | 0/8 [00:00<?, ?it/s]

{'num_projection_layers': 1, 'projection_dims': 128, 'dropout_rate': 0.1, 'vision_trainable': False, 'text_trainable': False, 'attention': True, 'lr': 0.001, 'epochs': 10, 'batch_size': 32}
Epoch 1/10
446/446 [==============================] - 120s 208ms/step - loss: 0.4790 - accuracy: 0.7883 - precision: 0.7164 - recall: 0.6365 - val_loss: 0.4602 - val_accuracy: 0.8346 - val_precision: 0.9397 - val_recall: 0.7871
Epoch 2/10
446/446 [==============================] - 57s 127ms/step - loss: 0.3078 - accuracy: 0.8746 - precision: 0.8324 - recall: 0.7957 - val_loss: 0.5246 - val_accuracy: 0.7400 - val_precision: 0.9164 - val_recall: 0.6446
Epoch 3/10
446/446 [==============================] - 57s 129ms/step - loss: 0.2658 - accuracy: 0.8926 - precision: 0.8580 - recall: 0.8241 - val_loss: 0.4445 - val_accuracy: 0.8383 - val_precision: 0.8972 - val_recall: 0.8384
Epoch 4/10
446/446 [==============================] - 58s 130ms/step - loss: 0.2417 - accuracy: 0.9045 - precision: 0.8772 - rec

 12%|█▎        | 1/8 [10:52<1:16:05, 652.27s/it]

{'num_projection_layers': 1, 'projection_dims': 128, 'dropout_rate': 0.1, 'vision_trainable': False, 'text_trainable': False, 'attention': True, 'lr': 0.0005, 'epochs': 10, 'batch_size': 32}
Epoch 1/10
446/446 [==============================] - 107s 200ms/step - loss: 0.4708 - accuracy: 0.7456 - precision: 0.7220 - recall: 0.5311 - val_loss: 0.5192 - val_accuracy: 0.8242 - val_precision: 0.9550 - val_recall: 0.7556
Epoch 2/10
446/446 [==============================] - 58s 129ms/step - loss: 0.3308 - accuracy: 0.8614 - precision: 0.8151 - recall: 0.7723 - val_loss: 0.5360 - val_accuracy: 0.8232 - val_precision: 0.9549 - val_recall: 0.7539
Epoch 3/10
446/446 [==============================] - 58s 130ms/step - loss: 0.2863 - accuracy: 0.8812 - precision: 0.8415 - recall: 0.8065 - val_loss: 0.7739 - val_accuracy: 0.5299 - val_precision: 0.8895 - val_recall: 0.2867
Epoch 4/10
446/446 [==============================] - 58s 131ms/step - loss: 0.2506 - accuracy: 0.9015 - precision: 0.8731 - re

 25%|██▌       | 2/8 [22:19<1:07:15, 672.66s/it]

{'num_projection_layers': 1, 'projection_dims': 128, 'dropout_rate': 0.2, 'vision_trainable': False, 'text_trainable': False, 'attention': True, 'lr': 0.001, 'epochs': 10, 'batch_size': 32}
Epoch 1/10
125/446 [=======>......................] - ETA: 58s - loss: 0.6397 - accuracy: 0.5994 - precision: 0.5882 - recall: 0.2947